In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os, tempfile, glob, random, json
from pathlib import Path
from IPython.display import Markdown
from PIL import Image
from getpass import getpass
import numpy as np
from itertools import combinations

import tiktoken #f

from uuid import uuid4
# Embedding model
from FlagEmbedding import BGEM3FlagModel

# LLM: openai and google_genai
import openai
from langchain_openai import OpenAI, OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# LLM: HuggingFace
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.llms import HuggingFaceHub

# LLM: Ollama
from langchain_ollama import ChatOllama

# langchain prompts, memory, chains...
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_community.chat_message_histories import StreamlitChatMessageHistory #f
from operator import itemgetter #f
from langchain_core.runnables import RunnablePassthrough #e
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_core.documents import Document
from langchain_core.prompts.base import format_document #f
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.messages import ChatMessage, HumanMessage #e
from langchain.embeddings.base import Embeddings

# Document loaders
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    DirectoryLoader,
    CSVLoader,
    UnstructuredExcelLoader,
    Docx2txtLoader,
    JSONLoader
)

# Text Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter

# OutputParser
from langchain_core.output_parsers import StrOutputParser

# Chroma: vectorstore
from langchain_community.vectorstores import Chroma

# Contextual Compression

#from langchain.retrievers import DocumentCompressorPipeline

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter,LongContextReorder

#from langchain.retrievers import EmbeddingsFilter
#from langchain.retrievers import ContextualCompressionRetriever

# Cohere
from langchain_community.llms import Cohere

### Creamos la clase necesaria para poder trabajar con el modelo de embedding

In [2]:
class BGEM3Langchain(Embeddings):
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        results = self.model.encode(texts, return_dense=True, return_sparse=False, return_colbert_vecs=False)
        # Solamente devolvemos el embedding por densidad, ya que, estos son los que funcionan en chromadb
        return results["dense_vecs"].tolist()

    def embed_query(self, text):
        result = self.model.encode([text], return_dense=True, return_sparse=False, return_colbert_vecs=False)
        return result["dense_vecs"][0].tolist()

### Cargamos el JSON CPEP

In [3]:
with open("../resources/cpe.json", "r", encoding="utf-8") as f:
    data = json.load(f)

docs = []

In [4]:
for item in data:
    tipo = item.get("tipo", "").capitalize()

    if tipo == "Introduccion":
        text = f"Título: {item.get('titulo', '')}\n" \
                f"Subtítulo: {item.get('subtitulo', '')}\n\n" \
                f"{item.get('contenido', '')}"

    elif tipo == "Articulo":
        text = (
            f"Parte {item.get('parte_num', '')}: {item.get('parte_nom', '')}\n"
            f"Título {item.get('titulo_num', '')}: {item.get('titulo_nom', '')}\n"
            f"Capítulo {item.get('capitulo_num', '')}: {item.get('capitulo_nom', '')}\n"
            f"Sección {item.get('seccion_num', '')}: {item.get('seccion_nom', '')}\n"
            f"Artículo {item.get('art_num', '')}: {item.get('nombre_juridico', '')}\n\n"
            f"{item.get('contenido', '')}"
        )
        
    elif tipo == "Disposición":
        text = (
            f"Disposición {item.get('disposicion', '')}\n"
            f"Nombre jurídico: {item.get('nombre_juridico', '')}\n\n"
            f"{item.get('contenido', '')}"
        )

    # Pongo este else, porque sino me sale error en page_content=text, porque es posible que text este vacio
    else:
        text = item.get("contenido", "")

    docs.append(Document(page_content=text, metadata={"tipo": tipo}))

print(f"Se cargaron {len(docs)} documentos desde el JSON.")

Se cargaron 426 documentos desde el JSON.


In [5]:
import random
random_document_id = random.choice(range(len(docs)))

print("test: ", random_document_id)
print(docs[random_document_id])

test:  59
page_content='Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 2: Derechos Fundamentales y Garantías
Capítulo Quinto: Derechos sociales y económicos
Sección 5: Derechos de la niñez, adolescencia y juventud
Artículo 58: Definiciones de Niño, Niña y Adolescente

Se considera niña, niño o adolescente a toda persona menor de edad. Las niñas, niños y adolescentes son titulares de los derechos reconocidos en la Constitución, con los límites establecidos en ésta, y de los derechos específicos inherentes a su proceso de desarrollo; a su identidad étnica, sociocultural, de género y generacional; y a la satisfacción de sus necesidades, intereses y aspiraciones.' metadata={'tipo': 'Articulo'}


In [6]:
# Embedding Model
model_embedding = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)
embedding_adapter = BGEM3Langchain(model_embedding)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
LOCAL_VECTOR_STORE_DIR = Path("./data").resolve().parent.joinpath("data", "vector_stores")

In [7]:
def create_vectorstore(collection_name, model_embedding, vectorstore_name):
    persist_directory = ("./" + vectorstore_name)
    vector_store = Chroma(
        collection_name = collection_name,
        embedding_function=model_embedding,
        persist_directory = persist_directory
    )
    return vector_store, vectorstore_name

In [8]:
create_new_vectorstore = True # Cambiar a true si queremos volver a crear la bd vectorial
if create_new_vectorstore:
    vector_store_bgem3,_ = create_vectorstore(
        collection_name = "cpep_bgem3",
        model_embedding = embedding_adapter,
        vectorstore_name= "vectordb/cpep_bgem3"
    )
    print("vector_store_bgem3:",vector_store_bgem3._collection.count(),"documentos.")

C:\Users\mauri\AppData\Local\Temp\ipykernel_10236\739684382.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_store = Chroma(


vector_store_bgem3: 0 documentos.


In [10]:
# Ids aleatorios evita colisiones, pero no tiene significado semantico
uuids = [str(uuid4()) for _ in range(len(docs))]

vector_store_bgem3.add_documents(documents=docs, ids=uuids)
print("vector_store_bgem3:",vector_store_bgem3._collection.count(),"documentos.")

NameError: name 'vector_store_bgem3' is not defined

In [ ]:
ids = []
metas = []
cont = 0

for i in range(len(docs)):
    if (i<2):
        ids.append(f"introduccion_{i+1}")
        metas.append({"tipo": "introduccion", "número": i+1})
    elif(i<399):
        ids.append(f"artículo_{i-1}")
        metas.append({"tipo": "artículo", "número": i-1})
    elif(i==399):
        ids.append(f"artículo_398_A")
        metas.append({"tipo": "artículo", "número": "398_A"})
    elif(i==400):
        ids.append(f"artículo_398_B")
        metas.append({"tipo": "artículo", "número": "398_B"})
    elif(i<414):
        ids.append(f"artículo_{i-2}")
        metas.append({"tipo": "artículo", "número": i-2})
    else:
        cont += 1
        ids.append(f"disposición_{cont}")
        metas.append({"tipo": "disposicion", "número": cont})

assert len(docs) == len(ids) == len(metas)

In [10]:
vector_store_bgem3.add_documents(documents=docs, ids=ids)
print("vector_store_bgem3:",vector_store_bgem3._collection.count(),"documentos.")

pre tokenize: 100%|██████████| 2/2 [00:00<00:00, 36.65it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 2/2 [00:03<00:00,  1.55s/it]


vector_store_bgem3: 426 documentos.


In [11]:
print(vector_store_bgem3._collection.count())
print(vector_store_bgem3._collection.peek())

426
{'ids': ['introduccion_1', 'introduccion_2', 'artículo_1', 'artículo_2', 'artículo_3', 'artículo_4', 'artículo_5', 'artículo_6', 'artículo_7', 'artículo_8'], 'embeddings': array([[ 0.04119873,  0.03131104, -0.01198578, ...,  0.01422882,
        -0.00476074, -0.02758789],
       [ 0.037323  ,  0.00144005, -0.04879761, ...,  0.00862885,
         0.00226021, -0.008255  ],
       [ 0.05194092, -0.00133228, -0.01771545, ...,  0.00419235,
         0.00993347, -0.03488159],
       ...,
       [ 0.05209351,  0.03167725, -0.04049683, ...,  0.00888824,
         0.04806519, -0.01847839],
       [ 0.03866577,  0.02000427, -0.02864075, ...,  0.01675415,
        -0.01379395, -0.02059937],
       [ 0.01986694, -0.00522232,  0.01384735, ...,  0.00788879,
         0.02572632, -0.037323  ]], shape=(10, 1024)), 'documents': ['Título: Antecedentes Legales\nSubtítulo: None\n\nLa Constitución Política del Estado, promulgada el 7 de febrero de 2009, consta de 5 Partes: I. BASES FUNDAMENTALES DEL ESTADO I

### Similarity Search

In [12]:
def print_documents(docs,search_with_score=False):
    if search_with_score:
        print(
            f"\n{'-' * 100}\n".join(
                [f"Document {i+1}:\n\n" + doc[0].page_content +"\n\nscore:"+str(round(doc[-1],3))+"\n" 
                for i, doc in enumerate(docs)]
            )
        )
    else:
        # used for similarity_search or max_marginal_relevance_search
        print(
            f"\n{'-' * 100}\n".join(
                [f"Document {i+1}:\n\n" + doc.page_content 
                for i, doc in enumerate(docs)]
            )
        )  

In [13]:
query = '¿Qué dice el artículo 1 de la constitución política del estado Boliviano?'
docs_withScores = vector_store_bgem3.similarity_search_with_score(query,k=5)

print_documents(docs_withScores,search_with_score=True)

Document 1:

Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 1: Bases fundamentales del estado
Capítulo Primero: Modelo de Estado
Sección None: None
Artículo 1: Modelol de Estado

Bolivia se constituye en un Estado Unitario Social de Derecho Plurinacional Comunitario, libre, independiente, soberano, democrático, intercultural, descentralizado y con autonomías. Bolivia se funda en la pluralidad y el pluralismo político, económico, jurídico, cultural y lingüístico, dentro del proceso integrador del país.

score:0.77

----------------------------------------------------------------------------------------------------
Document 2:

Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 1: Bases fundamentales del estado
Capítulo Segundo: Principios, Valores y Fines del Estado
Sección None: None
Artículo 10: Carácter Pacifista del Estado

I. Bolivia es un Estado pacifista, que promueve la cultura de la paz y el derecho a la paz, a

In [14]:
query_embeddings = embedding_adapter.embed_query(query)
docs_embeddings = embedding_adapter.embed_documents(
    [docs_withScores[i][0].page_content 
    for i in range(len(docs_withScores))
    ]
)

for i in range(len(docs_embeddings)):
    dot_product = round(np.dot(query_embeddings, docs_embeddings[i]),4)
    print(f"Similarty of document_{i} to the query: {dot_product}")

Similarty of document_0 to the query: 0.6149
Similarty of document_1 to the query: 0.5992
Similarty of document_2 to the query: 0.5826
Similarty of document_3 to the query: 0.5797
Similarty of document_4 to the query: 0.5777


### Maximum marginal relevance search (MMR) search

In [15]:
query = '¿Qué dice el artículo 1 de la constitución política del estado Boliviano?'
docs_MMR = vector_store_bgem3.max_marginal_relevance_search(query,k=4)

print_documents(docs_MMR)

Document 1:

Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 1: Bases fundamentales del estado
Capítulo Primero: Modelo de Estado
Sección None: None
Artículo 1: Modelol de Estado

Bolivia se constituye en un Estado Unitario Social de Derecho Plurinacional Comunitario, libre, independiente, soberano, democrático, intercultural, descentralizado y con autonomías. Bolivia se funda en la pluralidad y el pluralismo político, económico, jurídico, cultural y lingüístico, dentro del proceso integrador del país.
----------------------------------------------------------------------------------------------------
Document 2:

Parte Segunda: Estructura y organización funcional del estado
Título 7: Fuerzas Armadas y Policía Boliviana
Capítulo Primero: Fuerzas Armadas
Sección None: None
Artículo 249: Servicio Militar Obligatorio

Todo boliviano estará obligado a prestar servicio militar, de acuerdo con la ley.
--------------------------------------------------------

## Retrievers

### Vectorstore-backed retriever

In [16]:
def Vectorstore_backed_retriever(vectorstore,search_type="similarity",k=4,score_threshold=None):
    """create a vectorsore-backed retriever
    Parameters: 
        search_type: Defines the type of search that the Retriever should perform.
            Can be "similarity" (default), "mmr", or "similarity_score_threshold"
        k: number of documents to return (Default: 4) 
        score_threshold: Minimum relevance threshold for similarity_score_threshold (default=None)
    """
    search_kwargs={}
    if k is not None:
        search_kwargs['k'] = k
    if score_threshold is not None:
        search_kwargs['score_threshold'] = score_threshold

    retriever = vectorstore.as_retriever(
        search_type=search_type,
        search_kwargs=search_kwargs
    )
    return retriever

In [149]:
# similarity search
base_retriever_bgem3 = Vectorstore_backed_retriever(vector_store_bgem3, "similarity", k=3)
#base_retriever_OpenAI = Vectorstore_backed_retriever(vector_store_OpenAI,"similarity",k=10)
#base_retriever_google = Vectorstore_backed_retriever(vector_store_google,"similarity",k=10)
#base_retriever_HF = Vectorstore_backed_retriever(vector_store_HF,"similarity",k=10)

# # MMMR seach
# base_retriever_OpenAI = Vectorstore_backed_retriever(vector_store_OpenAI,"mmr",k=10)
# base_retriever_google = Vectorstore_backed_retriever(vector_store_google,"mmr",k=10)
# base_retriever_HF = Vectorstore_backed_retriever(vector_store_HF,"mmr",k=10)

# # similarity_score_threshold search
# base_retriever_OpenAI = Vectorstore_backed_retriever(vector_store_OpenAI,"similarity_score_threshold",score_threshold=0.5,k=10)
# base_retriever_google = Vectorstore_backed_retriever(vector_store_google,"similarity_score_threshold",score_threshold=0.5,k=10)
# base_retriever_HF = Vectorstore_backed_retriever(vector_store_HF,"similarity_score_threshold",score_threshold=0.5,k=10)

In [150]:
# Get relevant documents
query = '¿Qué dice el artículo 1 de la constitución política del estado Boliviano?'
relevant_docs = base_retriever_bgem3.invoke(query)

print_documents(relevant_docs)

Document 1:

Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 1: Bases fundamentales del estado
Capítulo Primero: Modelo de Estado
Sección None: None
Artículo 1: Modelol de Estado

Bolivia se constituye en un Estado Unitario Social de Derecho Plurinacional Comunitario, libre, independiente, soberano, democrático, intercultural, descentralizado y con autonomías. Bolivia se funda en la pluralidad y el pluralismo político, económico, jurídico, cultural y lingüístico, dentro del proceso integrador del país.
----------------------------------------------------------------------------------------------------
Document 2:

Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 1: Bases fundamentales del estado
Capítulo Segundo: Principios, Valores y Fines del Estado
Sección None: None
Artículo 10: Carácter Pacifista del Estado

I. Bolivia es un Estado pacifista, que promueve la cultura de la paz y el derecho a la paz, así como la co

## ChatModel

In [131]:
# LLM Model
llm = ChatOllama(model="mistral:7b-instruct-v0.2-q8_0")

In [133]:
import torch
# Message test
messages = [
    ChatMessage(role="control", content="thinking"),
    HumanMessage("¿Qué es la inteligencia artificial?"),
]


torch.cuda.empty_cache()
response = llm.invoke("¿Qué es la inteligencia artificial?")
print(response.content)

 La Inteligencia Artificial (IA) o "Inteligencia Machine" es una rama de la computación que se dedica al diseño y desarrollo de sistemas informáticos capaces de realizar tareas que hasta ahora solamente podían hacerlos los seres humanos con gran eficacia. Estas tareas incluyen aprender de experiencia, reconocer patrones y procesar lenguaje natural, entre otras.

El objetivo de la IA es crear sistemas que puedan entender y responder a un entorno complejo y cambiante, similar a cómo lo hacemos los seres humanos. Esto se logra mediante el uso de algoritmos y técnicas matemáticas como aprendizaje profundo, aprendizaje automático, lógica matemática y optimización.

La IA se utiliza en una variedad de aplicaciones, desde juegos como Go y Ajedrez, hasta sistemas de reconocimiento facial, asistencia médica, comercio electrónico y muchos más. El campo de la IA está en constante evolución y ofrece grandes oportunidades para el desarrollo de nuevas tecnologías que pueden transformar la manera en 

### Memory

In [80]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain_community.chat_models import ChatOllama

In [105]:
checkpointer = InMemorySaver()

In [106]:
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

In [107]:
builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")
builder.add_edge("call_model", END)

In [108]:
graph = builder.compile(checkpointer=checkpointer)

In [109]:
config = {"configurable": {"thread_id": "chat_temporal"}}

In [28]:
graph.invoke(
    {"messages": [{"role": "user", "content": "Hola, ¿quién eres?"}]},
    config
)

{'messages': [HumanMessage(content='Hola, ¿quién eres?', additional_kwargs={}, response_metadata={}, id='54ae8f3f-cb04-45a7-8f89-2c07bf12d1db'),
  AIMessage(content=' Hola! Soy un asistente de IA programado por Mistral AI. ¿Cómo puedo ayudarte hoy?\n\nTengo la capacidad para respondete preguntas, organizar tareas, proporcionar información y hacer muchas cosas más. ¡Vamos a trabajar juntos!', additional_kwargs={}, response_metadata={'model': 'mistral:7b', 'created_at': '2025-10-22T23:04:48.6278032Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11343189700, 'load_duration': 7909712300, 'prompt_eval_count': 13, 'prompt_eval_duration': 1974984600, 'eval_count': 79, 'eval_duration': 1443619000, 'model_name': 'mistral:7b', 'model_provider': 'ollama'}, id='lc_run--c9e90275-769e-4c8f-a61e-d23eaef8a093-0', usage_metadata={'input_tokens': 13, 'output_tokens': 79, 'total_tokens': 92})]}

In [29]:
graph.invoke(
    {"messages": [{"role": "user", "content": "¿Qué te dije antes?"}]},
    config
)

{'messages': [HumanMessage(content='Hola, ¿quién eres?', additional_kwargs={}, response_metadata={}, id='54ae8f3f-cb04-45a7-8f89-2c07bf12d1db'),
  AIMessage(content=' Hola! Soy un asistente de IA programado por Mistral AI. ¿Cómo puedo ayudarte hoy?\n\nTengo la capacidad para respondete preguntas, organizar tareas, proporcionar información y hacer muchas cosas más. ¡Vamos a trabajar juntos!', additional_kwargs={}, response_metadata={'model': 'mistral:7b', 'created_at': '2025-10-22T23:04:48.6278032Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11343189700, 'load_duration': 7909712300, 'prompt_eval_count': 13, 'prompt_eval_duration': 1974984600, 'eval_count': 79, 'eval_duration': 1443619000, 'model_name': 'mistral:7b', 'model_provider': 'ollama'}, id='lc_run--c9e90275-769e-4c8f-a61e-d23eaef8a093-0', usage_metadata={'input_tokens': 13, 'output_tokens': 79, 'total_tokens': 92}),
  HumanMessage(content='¿Qué te dije antes?', additional_kwargs={}, response_metadata={}, id='1ab7c35

In [31]:
# Volver a llamar para limpiar la memoria del chat
checkpointer = InMemorySaver()

### PromptTemplate

In [33]:
standalone_question_template = """
Dada la siguiente conversación y una pregunta de seguimiento,
reformula la pregunta de seguimiento para que sea una pregunta independiente,
manteniendo su significado original y en el mismo idioma (español).

Historial de chat:
{chat_history}

Pregunta de seguimiento:
{question}

Pregunta independiente:
"""

standalone_question_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template=standalone_question_template
)

In [ ]:
graph.invoke(
    {"messages": [{"role": "user", "content": "¿Qué dice el artículo 7 de la constitucion politica del estado de Bolivia?"}]},
    config
)

nueva_pregunta = "¿Y qué pasa si no se respetan esos derechos?"

state = graph.get_state(config)
historial = state.values["messages"]

prompt_reformulado = standalone_question_prompt.format(
    chat_history="\n".join([f"{m.type}: {m.content}" for m in historial]),
    question=nueva_pregunta
)

print("Pregunta reformulada:")
print(prompt_reformulado)

Pregunta reformulada:

Dada la siguiente conversación y una pregunta de seguimiento,
reformula la pregunta de seguimiento para que sea una pregunta independiente,
manteniendo su significado original y en el mismo idioma (español).

Historial de chat:
human: ¿Qué dice el artículo 7 de la constitucion politica del estado de Bolivia?
ai:  El Artículo 7 de la Constitución Política del Estado Plurinacional de Bolivia establece los principios fundamentales del Estado boliviano:

1. El Estado es sujeto de derecho, independiente, unitario, centralizado y democrático con personalidad jurídica propia.
2. La soberanía popular es indivisible, inalienable e inviolable.
3. La base del Estado es la nación boliviana plurinacional, integrada por todos los pueblos y nacionalidades de Bolivia.
4. El Estado tiene como objeto el desarrollo integral de la personalidad colectiva de la nación boliviana plurinacional, basado en la igualdad, justicia social, libertad, solidaridad, democracia, paz, concordia y u

In [51]:
respuesta_reformulada = llm.invoke(prompt_reformulado)

print("Pregunta generada por el propio modelo:\n")
print(respuesta_reformulada.content)

Pregunta generada por el propio modelo:

 ¿Qué sanciones legales implicarían la violación de los derechos establecidos en el Artículo 7 de la Constitución Política del Estado Plurinacional de Bolivia?


In [136]:
#Para borrar memoria del hilo
thread_id = "chat_temporal"
checkpointer.delete_thread(thread_id)

### ChatPromptTemplate

In [54]:
def answer_template(language="spanish"):
    template = f"""Eres un asistente especializado en la Constitución Política del Estado Plurinacional de Bolivia.
Responde la siguiente pregunta utilizando únicamente la información proporcionada en el contexto (delimitado por <context></context>). \
Tu respuesta debe estar en el idioma del final. 
Si la información no es suficiente o no se encuentra en el contexto, responde claramente: 
"No tengo información suficiente en la Constitución para responder a esta pregunta."

<context>
{{chat_history}}

{{context}} 
</context>

Pregunta: {{question}}

Idioma: {language}.
"""
    return template

In [55]:
chat_template = answer_template(language="Spanish")

In [ ]:
# En caso de no definirlo antes
config = {"configurable": {"thread_id": "chat_temporal"}}

In [ ]:
state = graph.get_state(config)
historial = state.values["messages"]
chat_history = "\n".join([f"{m.type}: {m.content}" for m in historial])
contexto = "\n".join([f"{m.type}: {m.content}" for m in historial]) # En este caso defino lo mismo para no dejarlo vacio y evitar errores, pero usualmente es el retrieved

In [57]:
pregunta_final = "¿Qué sucede si no se respetan los derechos fundamentales establecidos en el artículo 7?"

In [ ]:
prompt_final = chat_template.format(
    context = contexto,
    chat_history = chat_history,
    question = pregunta_final
)

In [63]:
print("prompt enviado:")
print(prompt_final)

PROMPT FINAL ENVIADO:
Eres un asistente especializado en la Constitución Política del Estado Plurinacional de Bolivia.
Responde la siguiente pregunta utilizando únicamente la información proporcionada en el contexto (delimitado por <context></context>). Tu respuesta debe estar en el idioma del final. 
Si la información no es suficiente o no se encuentra en el contexto, responde claramente: 
"No tengo información suficiente en la Constitución para responder a esta pregunta."

<context>
human: ¿Qué dice el artículo 7 de la constitucion politica del estado de Bolivia?
ai:  El Artículo 7 de la Constitución Política del Estado Plurinacional de Bolivia establece los principios fundamentales del Estado boliviano:

1. El Estado es sujeto de derecho, independiente, unitario, centralizado y democrático con personalidad jurídica propia.
2. La soberanía popular es indivisible, inalienable e inviolable.
3. La base del Estado es la nación boliviana plurinacional, integrada por todos los pueblos y na

In [64]:
respuesta = llm.invoke(prompt_final)
print("Respuesta:")
print(respuesta.content)

RESPUESTA DEL MODELO:
 Si no se respetan los derechos fundamentales establecidos en el artículo 7 de la Constitución Política del Estado Plurinacional de Bolivia, se violarían las bases y principios que rigen al Estado boliviano, lo que podría derivar en acciones legales o constitucionales correspondientes. En este caso, es importante señalar que la Constitución es la máxima norma jurídica del Estado boliviano y sus disposiciones tienen vigencia obligatoria para todos y cada uno de sus ciudadanos (artículo 7.11).


In [115]:
#Para borrar memoria del hilo
thread_id = "chat_temporal"
checkpointer.delete_thread(thread_id)

## Conversational Retrieval Chain

In [ ]:
from typing import List, Dict

class ConversationalRetrieval:
    def __init__(self, retriever, llm, memory, question_rewriter=None, answer_prompt=None, config=None):
        self.retriever = retriever
        self.llm = llm
        self.memory = memory
        self.question_rewriter = question_rewriter
        self.answer_prompt = answer_prompt
        self.config = config or {"configurable": {"thread_id": "default_thread"}}

    def condense_question(self, question: str, chat_history: List[Dict[str, str]]):
        if not self.question_rewriter:
            return question
        prompt = f"Reescribe la siguiente pregunta considerando el contexto previo:\n\nHistorial: {chat_history}\n\nPregunta: {question}"
        return self.question_rewriter.invoke(prompt).content

    def retrieve_context(self, question: str):
        return self.retriever.invoke(question)

    def generate_answer(self, question: str, context_docs: List[str], chat_history: List[str] = []):
        context_text = "\n".join([doc.page_content for doc in context_docs])
        chat_history_text = "\n".join(chat_history) if chat_history else "Sin historial previo."
        
        prompt = self.answer_prompt.format(
            context=context_text,
            chat_history=chat_history_text,
            question=question
        )
        
        return self.llm.invoke(prompt).content

    def invoke(self, question: str):
        try:
            state = self.memory.get_state(self.config)
            history_messages = state.values.get("messages", [])
            chat_history = [m.content for m in history_messages]
        except Exception:
            chat_history = []

        condensed_q = self.condense_question(question, chat_history)

        context_docs = self.retrieve_context(condensed_q)

        answer = self.generate_answer(condensed_q, context_docs)

        new_messages = [
            {"role": "user", "content": question},
            {"role": "ai", "content": answer}
        ]
        self.memory.update_state(self.config, {"messages": new_messages})

        return {"answer": answer, "source_documents": context_docs}


In [134]:
retriever = Vectorstore_backed_retriever(vector_store_bgem3, "similarity", k=5)
llm = ChatOllama(model="mistral:7b-instruct-v0.2-q8_0")
memory = graph
question_rewriter = llm
answer_prompt = chat_template

In [135]:
custom_chain = ConversationalRetrieval(
    retriever=retriever,
    llm=llm,
    memory=memory,
    question_rewriter=question_rewriter,
    answer_prompt=answer_prompt,
    config=config
)


In [152]:
#Para borrar memoria del hilo
thread_id = "chat_temporal"
checkpointer.delete_thread(thread_id)

In [157]:
graph.get_state(config) 

StateSnapshot(values={'messages': [HumanMessage(content='¿Qué dice la constitución política del estado Boliviano acerca de los pueblos indígenas?', additional_kwargs={}, response_metadata={}, id='54769969-764a-4df2-9df1-310bc7fa40a9'), AIMessage(content=' Respuesta en español:\n\nSegún el contexto anterior, con un registro histórico vacío, te proveeré información sobre la Constitución de Bolivia y sus disposiciones relacionadas con los pueblos indígenas.\n\nLa Constitución actual de Bolivia, adoptada en 2009, reconoce y respeta los derechos colectivos de los pueblos indígenas. Acknowledges their cultural diversity, their ancestral territories, and their right to autonomy and self-governance (Article 15). Además, garantiza la protección y promoción de sus idiomas, costumbres, tradiciones y espiritualidad (Artículo 6).\n\nLa Constitución también establece medidas de acción afirmativa para abordar las desigualdades históricas que han enfrentado los pueblos indígenas. Esto incluye disposic

In [155]:
response_first = custom_chain.invoke("¿Qué dice la constitución política del estado Boliviano acerca de los pueblos indígenas?")

In [156]:
print(response_first['source_documents'])

[Document(metadata={'tipo': 'Articulo'}, page_content='Parte Primera: Bases fundamentales del estado derechos, deberes y garantías\nTítulo 2: Derechos Fundamentales y Garantías\nCapítulo Primero: Disposiciones Generales\nSección None: None\nArtículo 13: Principios de Integralidad, Indivisibilidad y Universalidad de los Derechos\n\nI. Los derechos reconocidos por esta Constitución son inviolables, universales, interdependientes, indivisibles y progresivos. El Estado tiene el deber de promoverlos, protegerlos y respetarlos. II. Los derechos que proclama esta Constitución no serán entendidos como negación de otros derechos no enunciados. III. La clasificación de los derechos establecida en esta Constitución no determina jerarquía alguna ni superioridad de unos derechos sobre otros. IV. Los tratados y convenios internacionales ratificados por la Asamblea Legislativa Plurinacional, que reconocen los derechos humanos y que prohíben su limitación en los Estados de Excepción prevalecen en el o

In [158]:
print(response_first['answer'])

 Respuesta en español:

Según el contexto anterior, con un registro histórico vacío, te proveeré información sobre la Constitución de Bolivia y sus disposiciones relacionadas con los pueblos indígenas.

La Constitución actual de Bolivia, adoptada en 2009, reconoce y respeta los derechos colectivos de los pueblos indígenas. Acknowledges their cultural diversity, their ancestral territories, and their right to autonomy and self-governance (Article 15). Además, garantiza la protección y promoción de sus idiomas, costumbres, tradiciones y espiritualidad (Artículo 6).

La Constitución también establece medidas de acción afirmativa para abordar las desigualdades históricas que han enfrentado los pueblos indígenas. Esto incluye disposiciones relacionadas con la educación, el cuidado de salud y el desarrollo económico. Además, establece una Asamblea Consultiva Indígena Nacional como mecanismo de su participación en los procesos tomados todecía decisiones políticas (Artículo 16).

Por lo tanto,

In [159]:
response = custom_chain.invoke("Estas seguro que esos artículos hablan de esos temas?")

In [160]:
print(response["answer"])

 Respuesta en español: 

Sí, según el contexto proporcionado anteriormente, puedo confirmar que los Artículos 15 y 16 de la Constitución Boliviana abordan temas relacionados con derechos indígenas, diversidad cultural, territorios ancestrales, autonomía y autogobierno, protección de idiomas, costumbres, tradiciones, espiritualidad, acciones afirmativas, educación, atención sanitaria, desarrollo económico y su participación en decisiones políticas.

Idioma original: English.

English response: 

Answer in English:

Yes, based on the provided context earlier, Articles 15 and 16 of the Bolivian Constitution do cover topics related to indigenous peoples' rights, cultural diversity, ancestral territories, autonomy and self-governance, language protection, customs, traditions, spirituality, affirmative actions, education, health care, economic development, and their participation in political decisions.
